In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"

def init_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape all bus routes
def scrap_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape each bus details
def scrap_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allowing the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(8)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(6)  # Wait for the page to load more content

            # extract bus items details
            bus_name_item = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
            bus_type_item = driver.find_elements(By.CSS_SELECTOR, ".bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_item = driver.find_elements(By.CSS_SELECTOR, ".dp-time.f-19.d-color.f-bold")
            duration_item = driver.find_elements(By.CSS_SELECTOR, ".dur.l-color.lh-24")
            reaching_time_item = driver.find_elements(By.CSS_SELECTOR, ".bp-time.f-19.d-color.disp-Inline")
            star_rating_item = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_item = driver.find_elements(By.CSS_SELECTOR, ".fare.d-block")

            # Using XPath to handle both seat availability classes
            seat_availability_item = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_item)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_item[i].text,
                    "Bus_Type": bus_type_item[i].text,
                    "Departing_Time": departing_time_item[i].text,
                    "Duration": duration_item[i].text,
                    "Reaching_Time": reaching_time_item[i].text,
                    "Star_Rating": star_rating_item[i].text if i < len(star_rating_item) else '0',
                    "Price": price_item[i].text,
                    "Seat_Availability": seat_availability_item[i].text if i < len(seat_availability_item) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrap_all_pages(driver):
    for page in range(1, 6):  # There are 5 pages
        try:
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrap_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrap_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Main execution block
try:
    # Initialize driver once
    driver = init_driver()
    
    # Scrape routes and details from all pages
    scrap_all_pages(driver)

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(all_bus_details)

    # Save the DataFrame to a CSV file
    df.to_csv('sbstc_bus_details.csv', index=False)

finally:
    # Close the driver once scraping is complete
    driver.quit()
